In [1]:
import carla 
import math 
import random 
import time 
import numpy as np
import cv2

# Connect the client and set up bp library and spawn points
client = carla.Client('localhost', 2000) 
world = client.get_world()
bp_lib = world.get_blueprint_library()  
spawn_points = world.get_map().get_spawn_points() 

In [46]:
for v in world.get_actors().filter('*vehicle*'):
    v.destroy()
for s in world.get_actors().filter('*sensor*'):
    s.destroy()

In [47]:
# Spawn ego vehicle
vehicle_bp = bp_lib.find('vehicle.tesla.model3') 
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# Move spectator behind vehicle to view
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)

In [48]:
import importlib
import sys
import car

In [49]:
 #import the module here, so that it can be reloaded.
importlib.reload(sys.modules['car.sensors'])
from car.sensors import CarEquipment, base_sensor_specification, model3_base_offsets, base_sensor_types, base_callbacks

In [50]:
equipment = CarEquipment(world, vehicle)

In [51]:
equipment.build_sensors('base', base_sensor_types[0], base_sensor_specification, model3_base_offsets)

In [52]:
equipment.base_sensors['front'].is_listening

False

In [39]:
equipment.toggle_sensors('base', 'start', equipment.sem_callback, base_callbacks)

In [ ]:
img = equipment.sensor_data['rear']

In [45]:
def layout_func(sensor_data):
    top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image'], sensor_data['inst_image']), axis=1)
    lower_row = np.concatenate((sensor_data['depth_image'], sensor_data['dvs_image'], sensor_data['opt_image']), axis=1)
    tiled = np.concatenate((top_row, lower_row), axis=0)
    return tiled

In [44]:
equipment.display_sensors('Base Segmentation', layout_func, equipment.sensor_data)

[[[ 35 142 107 255]
  [ 35 142 107 255]
  [ 35 142 107 255]
  ...
  [ 35 142 107 255]
  [ 35 142 107 255]
  [ 35 142 107 255]]

 [[ 35 142 107 255]
  [ 35 142 107 255]
  [ 35 142 107 255]
  ...
  [ 35 142 107 255]
  [ 35 142 107 255]
  [ 35 142 107 255]]

 [[ 35 142 107 255]
  [ 35 142 107 255]
  [ 35 142 107 255]
  ...
  [ 35 142 107 255]
  [ 35 142 107 255]
  [ 35 142 107 255]]

 ...

 [[142   0   0 255]
  [142   0   0 255]
  [142   0   0 255]
  ...
  [142   0   0 255]
  [142   0   0 255]
  [142   0   0 255]]

 [[142   0   0 255]
  [142   0   0 255]
  [142   0   0 255]
  ...
  [142   0   0 255]
  [142   0   0 255]
  [142   0   0 255]]

 [[142   0   0 255]
  [142   0   0 255]
  [142   0   0 255]
  ...
  [142   0   0 255]
  [142   0   0 255]
  [142   0   0 255]]]
[[[ 35 142 107 255]
  [ 35 142 107 255]
  [ 35 142 107 255]
  ...
  [ 35 142 107 255]
  [ 35 142 107 255]
  [ 35 142 107 255]]

 [[ 35 142 107 255]
  [ 35 142 107 255]
  [ 35 142 107 255]
  ...
  [ 35 142 107 255]
  [ 35 142 1

In [158]:
equipment.sensor_data.keys()

dict_keys(['rear'])